Import Packages

In [1]:

# Need torch to load the model
import torch
# Model that is being used
from model import EncoderDecoderWrapper, Encoder, Decoder

# Used to work with/make HTTP requests
import requests
# Used to send/transfer data over a network
import json
# Used to grab data from specified time periods
import datetime
# Used to sort through data
import pandas as pd

# 
import alpaca_trade_api as tradeapi
import threading
import time
import websocket
#from alpaca_trade_api import StreamConn
from alpaca_trade_api.common import URL
import asyncio
import matplotlib.pyplot as plt
#
from alpaca.data import StockHistoricalDataClient, TimeFrame
from alpaca.data.requests import StockQuotesRequest, StockBarsRequest

Load Model

In [2]:
# Load mode onto GPU if CUDA is avaliable else load onto CPU
if torch.cuda.is_available():
    model = torch.load("./PairsTradingModel")
else:
    model = torch.load("./PairsTradingModel", map_location=torch.device('cpu'))

In [3]:
model = torch.load("./PairsTradingModel", map_location=torch.device('cpu'))

Global Variables

In [4]:
# Keys needs to verify/authenticate API and identify self
# JSON parse the strings into a dictionaty
HEADERS = json.loads(open("./auth.txt", 'r').read())
ALPACA_KEY_ID = HEADERS['APCA-API-KEY-ID']
ALPACA_SECRET_KEY = HEADERS['APCA-API-SECRET-KEY']
# URL needed to take action in the api
BASE_URL = 'https://paper-api.alpaca.markets/'
# Handles orders 
ORDERS_URL = f'{BASE_URL}v2/orders'
# Tells API that all trades will be on paper
ALPACA_PAPER = True

api = tradeapi.REST(ALPACA_KEY_ID,ALPACA_SECRET_KEY, base_url=BASE_URL, api_version = 'v2')

Methods

In [5]:
# Method to place an order through alpaca api
def send_order(symbol, qty, side, type = 'market', time_in_force = 'gtc'):
	# Creates data structure to be sent over the network
	data ={
	'symbol': symbol,
	'qty': qty,
	'side': side,
	'type': type,
	'time_in_force': time_in_force,
	}
	# Sends/posts data(json) to website (ORDERS_URL), with extra context/headers/identification(headers)
	r = requests.post(ORDERS_URL, json=data, headers = HEADERS)
	print(r)
	return json.loads(r.content)

In [6]:
# Setting up own 
api = tradeapi.REST(ALPACA_KEY_ID,ALPACA_SECRET_KEY, base_url=BASE_URL)

Alpaca API Testing

In [7]:
# Instantiate a data client
data_client = StockHistoricalDataClient(ALPACA_KEY_ID, ALPACA_SECRET_KEY)

# Set start time
start_time = pd.to_datetime("2024-02-01").tz_localize('America/New_York')
# Stocks to get data from
request_params = StockBarsRequest(
    symbol_or_symbols=['HYMTF','HMC'],
    timeframe=TimeFrame.Day,
    start=start_time
    )

bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)

In [8]:
# Reverse the order of data so that most recent occurs first
d = bars_df['close'].iloc[::-1]

In [9]:
from sklearn.preprocessing import MinMaxScaler
import torch
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
# Load model in onto the CPU
model = torch.load("./PairsTradingModel", map_location=torch.device('cpu'))

# Normalization for the data points
scaler = MinMaxScaler(feature_range=(0,1))
# Start date of data, need to figure out how to make this base on current date
start_time = pd.to_datetime("2024-02-01").tz_localize('America/New_York')

# HMC data request parameters
request_params = StockBarsRequest(
    symbol_or_symbols=['TJX'],
    timeframe=TimeFrame.Day,
    start=start_time
)
# Request to alpaca
bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)
# Getting most recent 30 days of data
bars_df = bars_df[-30:]
# Converting series to a list
TJX_30 = torch.tensor(bars_df['close'].tolist())

# HYMTF data request parameters
request_params = StockBarsRequest(
    symbol_or_symbols=['DLTR'],
    timeframe=TimeFrame.Day,
    start=start_time
)
# Request to alpaca
bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)
bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)
# Getting most recent 30 days of data
bars_df = bars_df[-30:]
# Converting series to a list to a torch tensor
DLTR_30 = torch.tensor(bars_df['close'].tolist())
# Calculating spread between the two stocks
DLTR_TJX_SPREAD = TJX_30 - DLTR_30
# Normalizing the spread and outputing a tensor
input_data = torch.FloatTensor(scaler.fit_transform(DLTR_TJX_SPREAD.reshape(-1,1))).unsqueeze(0).to(device)
# Compute predictions of model
model.to(device).eval()
with torch.inference_mode():
    output_data = model(input_data)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [77]:
input_data

tensor([[[0.0000],
         [0.1001],
         [0.0938],
         [0.0568],
         [0.0859],
         [0.1461],
         [0.1281],
         [0.0512],
         [0.2514],
         [0.3011],
         [0.2521],
         [0.4146],
         [0.3478],
         [0.3930],
         [0.4539],
         [0.4244],
         [0.4606],
         [0.3325],
         [0.3067],
         [0.3482],
         [0.6410],
         [0.6840],
         [0.7434],
         [0.7800],
         [0.7916],
         [0.8207],
         [0.9518],
         [0.9795],
         [0.9458],
         [1.0000]]])

In [73]:
# Want to make it so that it allocates half of the capital to the strategy

tensor([-23.7500, -21.0700, -21.2400, -22.2300, -21.4500, -19.8400, -20.3200,
        -22.3800, -17.0200, -15.6900, -17.0000, -12.6500, -14.4400, -13.2300,
        -11.6000, -12.3900, -11.4200, -14.8500, -15.5400, -14.4300,  -6.5900,
         -5.4400,  -3.8500,  -2.8700,  -2.5600,  -1.7800,   1.7300,   2.4700,
          1.5700,   3.0200])

In [65]:
# Get data for the day using YAHOO FINANCE and place trades and allocate capital using alpaca, since we are working with daily data

symbol  timestamp                
DLTR    2024-05-07 00:00:00-04:00   NaN
        2024-05-08 00:00:00-04:00   NaN
        2024-05-09 00:00:00-04:00   NaN
        2024-05-10 00:00:00-04:00   NaN
        2024-05-13 00:00:00-04:00   NaN
        2024-05-14 00:00:00-04:00   NaN
        2024-05-15 00:00:00-04:00   NaN
        2024-05-16 00:00:00-04:00   NaN
        2024-05-17 00:00:00-04:00   NaN
        2024-05-20 00:00:00-04:00   NaN
        2024-05-21 00:00:00-04:00   NaN
        2024-05-22 00:00:00-04:00   NaN
        2024-05-23 00:00:00-04:00   NaN
        2024-05-24 00:00:00-04:00   NaN
        2024-05-28 00:00:00-04:00   NaN
        2024-05-29 00:00:00-04:00   NaN
        2024-05-30 00:00:00-04:00   NaN
        2024-05-31 00:00:00-04:00   NaN
        2024-06-03 00:00:00-04:00   NaN
        2024-06-04 00:00:00-04:00   NaN
        2024-06-05 00:00:00-04:00   NaN
        2024-06-06 00:00:00-04:00   NaN
        2024-06-07 00:00:00-04:00   NaN
        2024-06-10 00:00:00-04:00   NaN
      